In [9]:
import pickle
import re
from collections import defaultdict
import os

In [3]:
g = open('hmm_tagger_l.pickle', 'rb')
m = pickle.load(g)


In [5]:
#getting unlabeled segmented data

data = open("marathi_data/mr/mr.segmented.EOLmarked.txt", "r").read().split("E O L")
data = [re.split(' |\n', ss)[1:-1] for ss in data]

In [7]:
# data[:2]

In [10]:
tag_dist = defaultdict(lambda: defaultdict(lambda: 0))
for sidx, sent in enumerate(data[:10000]):
    tags = m.best_path(sent)
    assert(len(tags) == len(sent))
    for tidx, tag in enumerate(tags):
        tag_dist[sent[tidx]][tag] += 1
        
    

In [77]:
# list(tag_dist.items())[100:200]

In [ ]:
#taglist
tags = {"NOUN", "VERB", "PROPN", "ADJ", "ADV", "INTJ", \
        "ADP", "AUX", "CCONJ", "SCONJ", "DET", "NUM", "PART", "PRON", \
        "PUNCT", "SYM", "X", \
        "NSUFF", "VSUFF", "AJSUFF", "PREF"}


In [13]:
token_dist = defaultdict(lambda: defaultdict(lambda: 0))
for token in tag_dist:
    for tag in tag_dist[token]:
        token_dist[tag][token] = tag_dist[token][tag]

In [19]:
closed = dict()

In [69]:
# sorted(list(token_dist["PROPN"].items()), key = lambda x: x[1], reverse = True)[:200]

In [70]:
#PUNCT
import string
# x = sorted(list(token_dist["PUNCT"].items()), key = lambda x: x[1], reverse = True)[:200]
closed["PUNCT"] = [a for a in string.punctuation]+["।"]
# closed["PUNCT"]

In [54]:
#PRON
# print(sorted(list(token_dist["PRON"].items()), key = lambda x: x[1], reverse = True)[200:300])
closed["PRON"] = {"मी", "आम्ही", "आपण", "आपल्या", "आम्हाला",
                  "तुम्ही", "तु", "तू", "तुम", "तुम्हाला",
                  "तो", "ती", "ते", "त्या", "ति",
                  "हा", "ही", "हे", "ह्या","हि",
                  "ज्या", "जी",
                  "या"}

In [133]:
#PART
sorted(list(token_dist["PART"].items()), key = lambda x: x[1], reverse = True)[:100]
closed["PART"] = {"पण", "च", "नाही", "नये", "तरी", "न", "ही", "नको", "नका"}

In [155]:
#AUX
x = sorted(list(token_dist["AUX"].items()), key = lambda x: x[1], reverse = True)[:100]
closed["AUX"] = {a for (a, b) in x}

In [214]:

# sorted(list(token_dist["NUM"].items()), key = lambda x: x[1], reverse = True)[:100]
# closed["AUX"]

In [181]:
#CCONJ
sorted(list(token_dist["CCONJ"].items()), key = lambda x: x[1], reverse = True)[:10]
closed["CCONJ"] = {"आणि", "व", "या", "य", "परंतु", "मग", "पण"}

In [72]:
#SCONJ
sorted(list(token_dist["SCONJ"].items()), key = lambda x: x[1], reverse = True)[:10]
closed["SCONJ"] = {"की", "तर", "जर"}

In [76]:
#DET
det = set()
x = sorted(list(token_dist["DET"].items()), key = lambda x: x[1], reverse = True)[:20]
for (a, b) in x:
    print(a)
    y = input()
    if y=="1":
        det.add(a)
        
closed["DET"] = det
# closed["DET"]

In [221]:

closed= {}
directory = 'marathi_data/mr/tag_prior_knowledge/closed_class/'
for file in os.listdir(directory):
    if file.endswith(".txt"):
        new_class = set()
        with open(directory+file, 'r') as fn:
            new_class = set(fn.read().split('\n')).difference({"", " ", "\n"})
        closed[file[:-4]] = new_class


In [213]:
# INDICATORS

In [ ]:
# x = {'णं':'inf', 'णे':'inf',
#                  'तो':'pres p1 sg. masc.', 'ते':'pres p1 sg. fem.',
#                  ' ेस':'pres p2 sg. fem',' ोस':'pres p2 sg. masc.', 'ता':'pres p2 hon.',
#                  'तो':'pres p3 sg. masc.', 'ते':'pres p3 sg. fem.','तं':'pres p1 sg. neut.', 'तात':'pres p3 pl.',
#                  'लो':'past p1 sg. masc.', 'ले':'past p1 sg. fem.','लो':'past p1 pl.', 
#                  'लास':'past p2 sg. masc.', 'लीस':'past p2 sg. fem.', 'ला':'past p2 hon.',
#                  'ला':'past p3 sg. masc.','ली':'past p3 sg. fem','लं':'past p3 neut.', 
#                  'ले':'past p3 pl. masc.','ल्या':'past p3 pl. fem.',                 
#                  'णार':'fut',
#                  ' ीन':'fut p1 sg.', 'ू':'fut p1 pl.',
#                  'शील':'fut p2 sg.',' ाल':'fut p2 pl./hon.',
#                  ' ेल':'fut p3 sg.',' ील':'fut p3 sg.', 'तील':'fut p3 pl./hon.', 
#                  'ावा':'subj masc.','ावं':'subj neut.','ावी':'subj fem.',
#                  'त':'pof negation',
#                  'ऊ':'pof verb complex',' ू':'pof verb complex',
#                  'ताना':'cont.', 
#                  'ून':'completive suffix',
#                  'ायला':'intent suffix',
#                  'लेला':'participle sg. masc.','लेली':'participle sg. fem.','लेलं':'participle sg. neut.', 'लेले':'participle pl. masc.', 'लेल्या':'participle pl. fem.',
#                  'णारा':'partciple sg. masc.','णारी':'participle sg. fem.', 'णारं':'participle sg. neut.','णारे':'participle pl. masc.','णार्या':'participle pl. fem./acc',}

In [174]:
indicator = dict()
indicator["ADP"] = {"चा", "ची", "चे", "च्या", \
                   "ात", "वर", "खाली", "जवळ", "मध्ये",\
                   "ला", "नी", "तून", "हून"}
indicator["ADV"] = {"नंतर", "पूर्वी", "आधी", "ाधी", "मुळे", "सारख", "पासून", "पर्यंत", "वर", "साठी"}
indicator["AJSUFF"] = {"पणे", "पणा"}
indicator["NSUFF"] = {"े", "्या"} #plural suffixes
indicator["VSUFF"] = set(x.keys())
indicator["PREF"] = {"अ", "अनु", "वि", "पर", "सद", "बद", "दुर", "उप"}
suff_tag = {"NOUN":"NSUFF", "VERB":"VSUFF", "ADJ":"AJSUFF"}

In [ ]:
closed["AUX"] = closed["AUX"].difference(indicator["VSUFF"])

In [216]:
directory = "marathi_data/mr/tag_prior_knowledge/indicators/"
for c in indicator:
    with open(directory+c+".txt", "w") as f:
        for m in indicator[c]:
            print(m, file=f)
        f.close()

In [219]:
indicator= {}
directory = 'marathi_data/mr/tag_prior_knowledge/indicators/'
for file in os.listdir(directory):
    if file.endswith(".txt"):
        new_class = set()
        with open(directory+file, 'r') as fn:
            new_class = set(fn.read().split('\n')).difference({"", " ", "\n"})
        indicator[file[:-4]] = new_class



In [220]:
indicator

{'VSUFF': {' ाल',
  ' ीन',
  ' ील',
  ' ू',
  ' ेल',
  ' ेस',
  ' ोस',
  'ऊ',
  'णं',
  'णार',
  'णारं',
  'णारा',
  'णारी',
  'णारे',
  'णार्या',
  'णे',
  'त',
  'तं',
  'ता',
  'तात',
  'ताना',
  'तील',
  'ते',
  'तो',
  'लं',
  'ला',
  'लास',
  'ली',
  'लीस',
  'ले',
  'लेलं',
  'लेला',
  'लेली',
  'लेले',
  'लेल्या',
  'लो',
  'ल्या',
  'शील',
  'ायला',
  'ावं',
  'ावा',
  'ावी',
  'ू',
  'ून'},
 'ADP': {'खाली',
  'चा',
  'ची',
  'चे',
  'च्या',
  'जवळ',
  'तून',
  'नी',
  'मध्ये',
  'ला',
  'वर',
  'हून',
  'ात'},
 'ADV': {'आधी',
  'नंतर',
  'पर्यंत',
  'पासून',
  'पूर्वी',
  'मुळे',
  'वर',
  'साठी',
  'सारख',
  'ाधी'},
 'PREF': {'अ', 'अनु', 'उप', 'दुर', 'पर', 'बद', 'वि', 'सद'},
 'AJSUFF': {'पणा', 'पणे'},
 'NSUFF': {'े', '्या'}}

In [142]:
def correct_tag(morph, tag, main_tag = None):
    
    if morph.isnumeric():
        return "NUM"
    
    allowed = {"VERB":{"AUX", "VSUFF", "ADP", "PART"},"NOUN":{"NSUFF", "ADP", "PART"}}
    for cclass in closed:
        if morph in closed[cclass]:
            if main_tag==None or (main_tag and main_tag in allowed and cclass in allowed[main_tag]):
                return cclass
    
    if len(morph)==1:
        for cclass in indicator:
            for morph in indicator[cclass]:
                if main_tag==None or (main_tag and main_tag in allowed and cclass in allowed[main_tag]):
                    return cclass
        
    if len(morph)>1:    #fuzzy check for equality    
        for cclass in indicator:
            for aff in indicator[cclass]:
                if morph in aff or aff in morph and abs(len(morph) - len(aff))<=2:
                    if main_tag==None or (main_tag and main_tag in allowed and cclass in allowed[main_tag]):
                        return cclass
                    
    return tag
                

In [148]:
def update_tag_dist_l(morphs, tags, tag_dist):
    tagged = []
    morph_sent = " ".join(morphs).split(" ")
    for tidx, tag in enumerate(tags):
        morph = morphs[tidx].split(" ")
        
        if len(morph)==1:
            tag = correct_tag(morph[0], tag)
            if tag in closed and morph[0] not in closed[tag]:
                tagged.append("NONE")
                continue
            tag_dist[morph[0]][tag] += 1
            tagged.append(tag)
        if len(morph)>1:
            if morph[0] in indicator["PREF"]:
                tag_dist[morph[0]]["PREF"] += 1
                morph = morph[1:]
                tagged.append("PREF")
                
            if len(morph[0])!=1:
                tag = correct_tag(morph[0], tag)
                if tag in closed and morph[0] not in closed[tag]:
                    tagged+=["NONE"]*len(morph)
                    continue

                tag_dist[morph[0]][tag] += 1
                morph = morph[1:]
                tagged.append(tag)
            
            for midx, m in enumerate(morph):
                if correct_tag(m, tag, tag) != tag: #if the morph is in the closed classes/indicators
                    tag_dist[m][correct_tag(m, tag)] += 1
                    tagged.append(correct_tag(m, tag, tag))
                else: #otherwise, we put in suffix bucket
                    if tag in suff_tag:
                        tag_dist[m][suff_tag[tag]]+=1
                        tagged.append(suff_tag[tag])
                    else:
                        tag_dist[m][tag] += 1
                        tagged.append(tag)
    print("\n\nTAGGED")
    print(*list(zip(morph_sent, tagged)), sep = "\n")
                
    return tag_dist
                

In [108]:
def get_main_morph(tok):
#     print(tok)
    for aidx, a in enumerate(tok.split(" ")):
        if a not in indicator["PREF"]:
#             print(aidx)
            return aidx
    
    return 0
    

In [100]:
#unlabelled segmented data
data = open("marathi_data/mr/mr.segmented.EOLmarked.txt", "r").read().split("E O L\n")
sentences = [re.split(' |\n', ss)[:-1] for ss in data][:-1]
morphs = [sent.split("\n")[:-1] for sent in data] #token with morphs

In [149]:
tag_dist = defaultdict(lambda: defaultdict(lambda: 0))

for sidx, sent in enumerate(sentences[:20]):
    all_tags = m.best_path(sent)
#     print(len(sent))
#     print(sent)
    tag_ids = []
    tidx = 0
#     print(*morphs[sidx], sep="\n")
    for tok in morphs[sidx]:
        main_id = get_main_morph(tok)
        tag_ids.append(tidx+main_id)
#         print(tag_ids)
        tidx += len(tok.split(" "))
    tags = [all_tags[tidx] for tidx in tag_ids]
    print("INITIAL")
    print(*list(zip(morphs[sidx], tags)), sep="\n")
    assert(len(tags) == len(morphs[sidx]))
    tag_dist = update_tag_dist_l(morphs[sidx], tags, tag_dist)


INITIAL
('स्वप्न', 'NOUN')
('दाखवि णे', 'VERB')
('आणि', 'DET')
('आश्वासन े', 'NOUN')
('देणार े', 'NOUN')
('नेते ही', 'DET')
('लोक ांना', 'NOUN')
('खूप', 'NOUN')
('आवडत ात', 'NOUN')
('.', 'PUNCT')


TAGGED
('स्वप्न', 'NOUN')
('दाखवि', 'VERB')
('णे', 'VSUFF')
('आणि', 'CCONJ')
('आश्वासन', 'NOUN')
('े', 'ADP')
('देणार', 'NOUN')
('े', 'ADP')
('नेते', 'NONE')
('ही', 'NONE')
('लोक', 'NOUN')
('ांना', 'NSUFF')
('खूप', 'NOUN')
('आवडत', 'NOUN')
('ात', 'ADP')
('.', 'PUNCT')
INITIAL
('मुलां सोबतच', 'NOUN')
('पालक ांना', 'NOUN')
('सांभाळ ण्याची', 'VERB')
('जबाबदार ी', 'ADJ')
('ही', 'PART')
('विवाह ीत', 'NOUN')
('महिलां ची ही', 'PRON')
('आहे', 'AUX')
('.', 'PUNCT')
('.', 'PUNCT')
('.', 'PUNCT')
('के वळ', 'ADJ')
(',', 'PUNCT')
('विवाह', 'ADJ')
('झाला य', 'VERB')
('म्हणून', 'ADJ')
('वृद्ध', 'AJSUFF')
('पालक ांची', 'NOUN')
('जबाबदार ी', 'ADJ')
('त्यांना', 'PART')
('झटक ता', 'VERB')
('येणार', 'VERB')
('नाही', 'AUX')
('असं', 'PUNCT')
('हाय कोर्टा नं', 'DET')
('म्हटल ंय', 'VERB')
('.', 'PUNCT')


TAGGED
('

('आज च्या', 'NOUN')
('मेळाव्यात', 'DET')
('रामदास', 'PREF')
('कदम', 'PROPN')
(',', 'PROPN')
('अनंत', 'CCONJ')
('गीते', 'PRON')
('यांनी', 'ADP')
('आमच्या तले', 'NOUN')
('सर्व', 'ADJ')
('मतभेद', 'PART')
('आणि', 'PART')
('गैर समज', 'SCONJ')
('दूर', 'ADJ')
('झाले', 'VERB')
('असून', 'PUNCT')
('ज्या ंनी', 'PRON')
('आमच्या त', 'NOUN')
('भांड ं ण', 'NOUN')
('ला वली', 'AUX')
('त्यांना', 'DET')
('अम् ही', 'ADV')
('पळव ून', 'VERB')
('लावले', 'VERB')
('आहे', 'AUX')
('असं', 'PUNCT')
('सांग ताना', 'VERB')
('नारायण', 'CCONJ')
('राणें ना', 'PRON')
('लक्ष', 'NOUN')
('के लं', 'VERB')
('.', 'PUNCT')


TAGGED
('आज', 'NOUN')
('च्या', 'ADP')
('मेळाव्यात', 'NONE')
('रामदास', 'PREF')
('कदम', 'PROPN')
(',', 'PUNCT')
('अनंत', 'NONE')
('गीते', 'NONE')
('यांनी', 'ADP')
('आमच्या', 'NOUN')
('तले', 'NSUFF')
('सर्व', 'ADJ')
('मतभेद', 'NONE')
('आणि', 'CCONJ')
('गैर', 'NONE')
('समज', 'NONE')
('दूर', 'ADJ')
('झाले', 'VERB')
('असून', 'AUX')
('ज्या', 'PRON')
('ंनी', 'PRON')
('आमच्या', 'NOUN')
('त', 'ADP')
('भांड', 'NOUN')

In [91]:
sidx

50001